In [ ]:
import numpy as np
from astropy.stats import LombScargle
import scipy.signal as sig
import matplotlib.pyplot as plt
%matplotlib inline

# t is a linear time without reference to angular frequencies
# signal at frequency 1
def signal(N, cycles, noise):
    """
    :param N: number of samples in output sequence y
    :param cycles: number of cycles of signal to appear in sequence
    :param noise: amplitude of noise (signal has amplitude 1)
    :returns: t, y = arrays containing time, signal sequences, with 
    signal appearing at frequency 1
    """
    t = np.linspace(0, cycles, N)
    y = np.sin(2* np.pi * t) + np.random.rand(N) * noise
    return t, y

def fft(t, y):
    """
    :param t: linear time array
    :param y: signal sequence
    :returns: f, P = arrays containing (positive) frequencies of FFT and corresponding
    power spectrum (frequency in cycles / unit time)
    """
    N = len(t)
    f = np.fft.fftfreq(N, t[1]-t[0]) # this computes frequency w.r.t Delta t
    i = (f>0)
    f = f[i]
    P = abs(np.fft.fft(y)[i]) ** 2 / N
    return f, P

## Power spectrum

The 'psd' normalization returns indead the same power as a corresponding FFT:

In [ ]:
N = 256
C = 16
t, y = signal(N, C, 1)
plt.plot(t, y)

In [ ]:
f, P = fft(t, y)
LS = LombScargle(t, y).power(f, normalization='psd')
plt.figure(figsize=(10, 5))
plt.subplot(1,2,1)
plt.plot(f, P)
plt.subplot(1,2,2)
plt.plot(f, LS)

The forward fft is not normalized (https://docs.scipy.org/doc/numpy-1.12.0/reference/routines.fft.html), so that the inverse transform needs to be normalized by $\frac{1}{N}$. To go from power $P(f)$ to amplitude $A(f)$, one has  
$$A(f) = 2 \left(\frac{P(f)}{N} \right)^{1/2} $$
where the factor $2$ arises from the fact that we ignore the negative frequencies in the power spectrum (negative and positive frequencies add up to the full real amplitude when performing an inverse transform).

In [ ]:
p = LombScargle(t, y).power(1, normalization='psd')
2 * np.sqrt(p/N)

The equivalent result from fft is obtained when chosing the $C^{th}$ element in the returned fft array, where $C$ is the number of cycles of the periodic signal:

In [ ]:
p = np.fft.fft(y)[C] # no conversion to power first here
2 * abs(p) / N

## Windowing

### "DFT-even" and symmetric windows
The *sym* option in SciPy's [window functions](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.signal.hann.html#scipy.signal.hann) produces, when set to *False* and the window length is even a "DFT even" (or "periodic") window, i.e. one with symmetry about a central point which attains the value 1, but with the first element set to 0. This is useful for DFTs because they have an implied periodicity: the result would be unchanged if an $(N+1)^{th}$ value identical to the $0^{th}$ value followed a sequence of $N$ (an even number) elements <cite data-cite="2031184/N92WUBJI"></cite>. SciPy's implementation sets the first element of the window to zero, while Harris does it with last one - I don't know why.

### Window coherent gain
The application of a window function reduces the *gain* of the resulting DFT, changing the amplitude estimates. The gain for a Hann window (a $cos^{\alpha}(x)$ window with $\alpha=2$ in Harris' terminology) is 0.5. Therefore the amplitude estimate needs to be multiplied by an additional factor of 2.

In [ ]:
def welch(t, y, n):
    """
    Welch's method computes the average of the power spectra of windowed sub-sequences that
    overlap by 50%. The window is a Hann window.
    :param t: time sequence
    :param y: signal sequence
    :param n: number of non-overlapping subsequences (total number: 2n-1)
    """
    N = len(y)
    w = N // n # width
    win = sig.hann(w, sym=False)
    d = w // 2
    m = np.mean([LombScargle(t[:w], y[j*d:j*d+w] * win).power(1, normalization='psd')
        for j in range(2 * n - 1)])
    return 4 * np.sqrt(m / w) # 2 x the non-windowed estimate 

[welch(t, y, n) for n in [1,2,4,8,16]]

As comparison, here the amplitude estimate from simple "period folding":

In [ ]:
x = y.reshape((16,-1)).mean(0)
(max(x) - min(x)) / 2

## Coherence
Coherence of a raw DFT is always 1. Only when averaging over windows is employed can coherence be estimated. 

[The correct definition is](https://www.dsprelated.com/freebooks/mdft/Coherence_Function.html):
$$
\hat{C}_{xy}(\omega_k) = \frac{\left| \left\{ \overline{X_m(\omega_k)} Y_m(\omega_k) \right\}_m \right|^2}
{\left\{ \left| X_m(\omega_k) \right|^2 \right\}_m \left\{ \left| Y_m(\omega_k) \right|^2 \right\}_m}
$$

where the overbar denotes complex conjugation and the curly brackets averaging over windows $m$. Note the respective order of taking averages and moduli. 